In [ ]:
# Python 3.13
# If needed in a fresh env:
#   pip install "pymc>=5.21" "arviz>=0.17" "numpy>=2" "pandas>=2.2"
from pytensor.tensor.variable import TensorVariable
import numpy as np
import pandas as pd
import pymc as pm
import pytensor.tensor as pt
import arviz as az

# ---- 1) Get synthetic data from your helpers ----
from bayes_tools.helpers.synthetic_data_helpers import (
    make_hierarchical_ou_dataset,
    aggregate_to_parent,
)

# Build OU-level monthly panel
df_ou = make_hierarchical_ou_dataset(
    n_regions=1,
    n_sites_per_region=3,
    n_ous_per_site=4,
    n_years=1,
    wave_months=(6, 12),
    wave_missing_prob=0.,
    seed=7,
)

# (Optional) aggregate to parent level (e.g., 'site' or 'region')
# df_site = aggregate_to_parent(df_ou, level="site")

# We'll model at the OU level:
df = df_ou.copy()

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df[df["survey_score"].notna()].groupby(["ou_code", "date"])[["survey_score", "productivity"]].mean()

In [ ]:
df[df["survey_score"].notna()].groupby(["ou_code", "date"])[["survey_score", "productivity"]].mean()

In [ ]:
df.isna().sum()

In [ ]:

# ---- 2) Basic feature prep ----
# indices
df = df_ou.copy()
df = df.sort_values(["ou_code", "date"]).reset_index(drop=True)
ou_codes = df["ou_code"].astype("category")
ou_idx = ou_codes.cat.codes.to_numpy().astype("int32")

# time index (month panel)
dates = pd.to_datetime(df["date"])
unique_months = np.sort(dates.unique())
month_to_idx = {m: i for i, m in enumerate(unique_months)}
time_idx = np.array([month_to_idx[d] for d in dates], dtype="int32")
T = len(unique_months)
G = ou_codes.cat.categories.size

# outcomes and predictors
y_raw = df["productivity"].to_numpy().astype("float64")
# log-transform to make noise closer to Gaussian / stabilize variance
y = np.log(y_raw).astype("float64")

x_obs = df["survey_score"].to_numpy().astype("float64")  # may contain NaN
n_resp = df["n_respondents"].to_numpy()                  # NaN where no survey
n_resp_filled = np.where(np.isnan(n_resp), 0.0, n_resp).astype("float64")
has_survey = ~np.isnan(x_obs)
idx_obs = np.flatnonzero(has_survey)

# standardization helpers (kept simple; you can swap for robust scaling)
def zscore(a: np.ndarray):
    m = np.nanmean(a)
    s = np.nanstd(a)
    s = s if s > 0 else 1.0
    return (a - m) / s, m, s

y_z, y_mean, y_sd = zscore(y)      # target
x_z = x_obs.copy()
if np.isfinite(x_obs[idx_obs]).any():
    x_z, x_mean, x_sd = zscore(x_obs)
else:
    # fallback if synthetic happens to miss every survey (unlikely)
    x_mean, x_sd = 0.0, 1.0
    x_z = (x_obs - x_mean) / x_sd

# ---- 3) PyMC model ----
with pm.Model() as model:
    # --- Global means for OU intercept/slope (on log-productivity scale) ---
    mu_alpha = pm.Normal("mu_alpha", 0.0, 1.0)
    mu_beta  = pm.Normal("mu_beta",  0.0, 0.5)

    # --- Correlated OU effects: [alpha, beta] via LKJ-Cholesky for robustness ---
    L_ou, corr_ab, sds_ab = pm.LKJCholeskyCov(
        "L_ou",
        n=2,
        eta=2.0,
        sd_dist=pm.HalfNormal.dist(1.0),
        compute_corr=True,
        store_in_trace=True,
    ) # type: ignore
    L_ou: TensorVariable = L_ou
    assert isinstance(L_ou, TensorVariable) and isinstance(corr_ab, TensorVariable) and isinstance(sds_ab, TensorVariable)
    z = pm.Normal("z", 0.0, 1.0, size=(G, 2))          # Gx2
    ab = pt.dot(z, L_ou.T)                              # Gx2
    alpha_ou = pm.Deterministic("alpha_ou", mu_alpha + ab[:, 0])
    beta_ou  = pm.Deterministic("beta_ou",  mu_beta  + ab[:, 1])

    # --- Time fixed effects (centered) for the outcome ---
    time_raw = pm.Normal("time_raw", 0.0, 1.0, shape=T)
    time_eff = time_raw - pt.mean(time_raw)

    # --- Latent survey process x* (z-scored scale) ---
    # OU-level mean for x*
    mu_x = pm.Normal("mu_x", 0.0, 1.0)
    sigma_mu_x = pm.HalfNormal("sigma_mu_x", 1.0)
    mu_x_ou = pm.Normal("mu_x_ou", mu_x, sigma_mu_x, shape=G)

    # Time effect for x* (optional; helps interpolate sparse waves)
    time_x_raw = pm.Normal("time_x_raw", 0.0, 1.0, shape=T)
    time_x_eff = time_x_raw - pt.mean(time_x_raw)

    # Latent x* per OU-month with residual variance
    sigma_x = pm.HalfNormal("sigma_x", 1.0)
    
    # gamma_time_x = pm.Normal("gamma_time_x", 0.0, 0.3)

    x_latent = pm.Normal(
        "x_latent",
        # mu_x_ou[ou_idx] + gamma_time_x * time_x_eff[time_idx],  # small/shrunk time wiggle
        mu_x_ou[ou_idx] + 0.2 * time_x_eff[time_idx],
        sigma_x,
        shape=y_z.shape[0],
    )

    # --- Measurement model where survey is observed ---
    # Downweight by sqrt(n_resp): larger n -> lower noise
    # Guard against n=0: clamp minimum effective n to 1.
    n_eff = pt.clip(pt.sqrt(pt.maximum(pt.as_tensor_variable(n_resp_filled), 1.0)), 1.0, 1000.0) # type: ignore
    sigma_meas_base = pm.HalfNormal("sigma_meas_base", 1.0)
    sigma_meas = sigma_meas_base / n_eff

    # Only impose likelihood where survey actually observed
    pm.Normal(
        "survey_obs",
        mu=x_latent[idx_obs],
        sigma=sigma_meas[idx_obs],
        observed=(x_z[idx_obs]),
    )

    # --- Outcome model: log productivity (z-scored) ---
    # Residual noise
    sigma_y = pm.HalfNormal("sigma_y", 0.5)

    mu_y = alpha_ou[ou_idx] + time_eff[time_idx] + beta_ou[ou_idx] * x_latent
    y_like = pm.Normal("y_like", mu=mu_y, sigma=sigma_y, observed=y_z)

    # --- Helpful deterministics for interpretation on original scales ---
    pm.Deterministic("corr_alpha_beta", corr_ab[0, 1])
    pm.Deterministic("beta_global", mu_beta)

     # ---- 4) Sample ----
    idata = pm.sample(
        draws=1000,
        tune=1000,
        chains=4,
        cores=4,
        target_accept=0.9,
        random_seed=7,
        progressbar=True,
    )

In [ ]:
x_post_mean = idata.posterior["x_latent"].mean(("chain","draw")).values
np.corrcoef(x_post_mean[idx_obs], x_z[idx_obs])[0,1]   # expect high

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, arviz as az

# Nice labels for OUs
ou_labels = pd.Index(pd.Categorical(df["ou_code"]).categories.astype(str), name="ou_code")

# Quick tables
summ_alpha = az.summary(idata, var_names=["alpha_ou"], hdi_prob=0.9)
summ_beta  = az.summary(idata, var_names=["beta_ou"],  hdi_prob=0.9)

# Attach OU codes to summaries (indices like 'beta_ou[0]')
def add_ou_labels(summ, ou_labels):
    lab = []
    for idx in summ.index:
        i = int(idx.split("[")[1].rstrip("]"))
        lab.append(ou_labels[i])
    summ = summ.assign(ou_code=lab).set_index("ou_code")
    return summ

summ_alpha = add_ou_labels(summ_alpha, ou_labels)
summ_beta  = add_ou_labels(summ_beta,  ou_labels)

print("Alpha per OU (posterior mean and 90% HDI):")
display(summ_alpha[["mean","hdi_5%","hdi_95%"]])
print("\nBeta per OU (posterior mean and 90% HDI):")
display(summ_beta[["mean","hdi_5%","hdi_95%"]])

# Forest plots
az.plot_forest(idata, var_names=["beta_ou"], combined=True, hdi_prob=0.9, figsize=(8, 0.35*len(ou_labels)+2))
plt.title("OU slopes β_g (x* → log-productivity, both z-scored)")
plt.show()

az.plot_forest(idata, var_names=["alpha_ou"], combined=True, hdi_prob=0.9, figsize=(8, 0.35*len(ou_labels)+2))
plt.title("OU intercepts α_g (log-productivity z-score)")
plt.show()

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, arviz as az
# assumes: idata, y_sd, x_sd, df, ou_labels already defined

# Pull posterior: dims ~ (chain, draw, beta_ou_dim_0)
beta = idata.posterior["beta_ou"]

# Map to % change in raw productivity per +1 survey point
mult = np.exp((y_sd / x_sd) * beta)
pct  = (mult - 1.0) * 100.0  # dims: (chain, draw, ou)

# Posterior mean & 90% interval over chain/draw
pct_mean = pct.mean(dim=("chain","draw")).values  # shape: (G,)
q = pct.quantile([0.05, 0.95], dim=("chain","draw"))          # dims: (quantile, ou)
pct_lo = q.sel(quantile=0.05).values
pct_hi = q.sel(quantile=0.95).values

eff_df = pd.DataFrame({
    "ou_code": pd.Index(pd.Categorical(df["ou_code"]).categories.astype(str)),
    "pct_mean": pct_mean, "pct_lo": pct_lo, "pct_hi": pct_hi
}).set_index("ou_code").sort_values("pct_mean")

display(eff_df)

# Plot interval per OU
fig, ax = plt.subplots(figsize=(8, 0.35*len(eff_df)+2))
ypos = np.arange(len(eff_df))
ax.hlines(y=ypos, xmin=eff_df["pct_lo"], xmax=eff_df["pct_hi"])
ax.plot(eff_df["pct_mean"], ypos, "o")
ax.axvline(0, ls="--", lw=1)
ax.set_yticks(ypos, eff_df.index)
ax.set_xlabel("% change in raw productivity per +1 survey point (90% interval)")
ax.set_title("OU-specific effect interpretation")
plt.tight_layout(); plt.show()

In [ ]:
# Quick tables
summ_alpha = az.summary(idata, var_names=["alpha_ou"], hdi_prob=0.9)
summ_beta  = az.summary(idata, var_names=["beta_ou"],  hdi_prob=0.9)

def add_ou_labels(summ, labels):
    labs = []
    for idx in summ.index:
        i = int(idx.split("[")[1].rstrip("]"))
        labs.append(labels[i])
    return summ.assign(ou_code=labs).set_index("ou_code")

ou_labels = pd.Index(pd.Categorical(df["ou_code"]).categories.astype(str), name="ou_code")
display(add_ou_labels(summ_alpha, ou_labels)[["mean","hdi_5%","hdi_95%"]])
display(add_ou_labels(summ_beta,  ou_labels)[["mean","hdi_5%","hdi_95%"]])

# Forest plots
az.plot_forest(idata, var_names=["beta_ou"], combined=True, hdi_prob=0.9,
               figsize=(8, 0.35*len(ou_labels)+2))
plt.title("OU slopes β_g (x* → log-productivity, both z-scored)"); plt.show()

az.plot_forest(idata, var_names=["alpha_ou"], combined=True, hdi_prob=0.9,
               figsize=(8, 0.35*len(ou_labels)+2))
plt.title("OU intercepts α_g (log-productivity z-score)"); plt.show()

In [ ]:
import matplotlib.dates as mdates

def plot_latent_for_ou(ou_code_str):
    labels = pd.Categorical(df["ou_code"]).categories.astype(str)
    g = int(np.where(labels == ou_code_str)[0][0])

    # rows for this OU, ordered in time
    row_idx_all = np.where(pd.Categorical(df["ou_code"]).codes == g)[0]
    t = time_idx[row_idx_all]
    order = np.argsort(t)
    row_idx = row_idx_all[order]
    months = pd.to_datetime(unique_months[t[order]])

    # posterior summaries for x_latent at those rows
    x_da  = idata.posterior["x_latent"]                     # (chain, draw, obs)
    x_mu  = x_da.mean(("chain","draw")).values[row_idx]
    qx    = x_da.quantile([0.05, 0.95], dim=("chain","draw"))
    lo    = qx.sel(quantile=0.05).values[row_idx]
    hi    = qx.sel(quantile=0.95).values[row_idx]

    # observed standardized survey at those rows
    obs_mask = has_survey[row_idx]
    obs_months = months[obs_mask]
    obs_vals   = x_z[row_idx][obs_mask]

    fig, ax = plt.subplots(figsize=(10,4))
    ax.fill_between(months, lo, hi, alpha=0.25, label="x* 90% interval")
    ax.plot(months, x_mu, lw=2, label="x* posterior mean")
    ax.scatter(obs_months, obs_vals, marker="x", s=60, label="observed survey (z)")
    ax.set_title(f"Latent survey x* over time — OU {ou_code_str}")
    ax.set_ylabel("Standardized survey (z)")
    ax.set_xlabel("Month")
    ax.legend()
    ax.xaxis.set_major_locator(mdates.AutoDateLocator())
    ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(ax.xaxis.get_major_locator()))
    plt.tight_layout(); plt.show()

# Example
plot_latent_for_ou(str(ou_labels[0]))

In [ ]:
# Quick plot: OU intercept vs slope post means
a_mu = idata.posterior["alpha_ou"].mean(("chain","draw")).to_numpy()
b_mu = idata.posterior["beta_ou"].mean(("chain","draw")).to_numpy()

plt.figure(figsize=(5,4))
plt.scatter(a_mu, b_mu)
for i, lab in enumerate(ou_labels):
    plt.annotate(str(lab), (a_mu[i], b_mu[i]), fontsize=8, xytext=(3,3), textcoords="offset points")
plt.axhline(0, lw=1, ls="--"); plt.axvline(0, lw=1, ls="--")
plt.xlabel("α_g (mean log-productivity z)"); plt.ylabel("β_g (slope)")
plt.title("OU intercepts vs slopes (posterior means)")
plt.tight_layout(); plt.show()

# Some single-parameter posteriors
az.plot_trace(idata, var_names=["beta_global","corr_alpha_beta","sigma_y","sigma_x","sigma_meas_base"], compact=True, figsize=(10,6))
plt.show()

In [ ]:
# import os
# os.environ["PATH"] = "/opt/homebrew/bin:" + os.environ["PATH"]
# # Optional: be explicit
# os.environ["GRAPHVIZ_DOT"] = "/opt/homebrew/bin/dot"
# pm.model_to_graphviz(model)

In [ ]:
g = pm.model_to_graphviz(model)
print(g.source[:1000])           # preview DOT text
g.save("model.dot")              # write DOT to file

In [ ]:
import shutil, os 
print ("dot:", shutil.which("dot"))

In [ ]:
import os, shutil
os.environ["PATH"] = "/opt/homebrew/bin:" + os.environ["PATH"]
print(shutil.which("dot"))

In [ ]:
g = pm.model_to_graphviz(model)
g.graph_attr.update(rankdir="LR")  # optional layout
g

In [ ]:

# ---- 5) Quick checks ----
az.summary(idata, var_names=[
    "mu_alpha", "mu_beta", "sigma_y", "sigma_x", "sigma_mu_x",
    "sigma_meas_base", "corr_alpha_beta"
], kind="stats")